# Project Dashboard Mark 5

In [1]:
print("Starting up!")
print("... Importing")
# Common imports from Python standard libraries
from dataclasses import dataclass
import math

# Common imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time

# Science imports
import scipy
from scipy import stats
from scipy import cluster
from statistics import mean
from sklearn.mixture import GaussianMixture
from sklearn.cluster import KMeans
from matplotlib import colors

# Useful imports for visualization
from plotly import subplots

# Plotly express and dash
import dash
from dash import dcc
from dash import html
from dash.dependencies import Input, Output
import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff


# Constants
EPS = 0.0000001
FINAL = False

# Some useful functions
def almost_equal(a,b,eps=EPS):
    return abs(a-b) <= eps

def binsearch(func, vv = 0.025, x_l = -100, x_r = 100, eps=0.00001):
    '''
    Search for x such that func(x) == vv,
    within a range of +- eps,
    with x between x_l and x_r.
    '''
    v_l = func(x_l)
    v_r = func(x_r)
    
    x_m = (x_l + x_r)/2
    v_m = func(x_m)
    
    while abs(v_m - vv) >= eps:
        if v_m < vv:
            x_l = x_m
        elif v_m > vv:
            x_r = x_m
        else:
            print("weird, this shouldn't be possible")
            return x_m
            
        v_l = func(x_l)
        v_r = func(x_r)
        x_m = (x_l + x_r)/2
        v_m = func(x_m)
    
    return x_m

# Code provided from HW4
def stdev(X):
    m = mean(X)
    return math.sqrt(sum((x-m)**2 for x in X) / len(X))

def degreesOfFreedom(X, Y):
    if len(X) == 0:
        return 1
    elif len(Y) == 0:
        return 1
    
    s1 = (stdev(X)**2)
    s2 = (stdev(Y)**2)
    df = (s1 / len(X) + s2 / len(Y))**2 / ((s1 / len(X))**2 / (len(X) - 1) + (s2 / len(Y))**2 /
(len(Y) - 1))
    return(df)

# # Things for part 5
FI = (1 + 5**.5)/2 # golden ratio

def get_hues(ii):
    hue = (FI * ii) % 1
    hue_int = round(hue * 255)
    return f"hsv({hue_int},255,128)"

def get_sample(array, n_samples = 15161):
    indices = np.random.choice(list(range(len(array))), size=n_samples)
    return array[indices]


df_full = pd.read_csv("../DATASETS/US_Accidents_Dec20_updated.csv")
df = df_full.sample(n = 10000, ignore_index = True)

app = dash.Dash("Traffic accident data studio")

Starting up!
... Importing


In [2]:
print("Building part 1...")

dist_checklist_keys = ['Stop', 'Traffic_Calming', 'Roundabout', 'Traffic_Signal', 'Crossing', 'Railway', 'Bump']
distance_by_severity = {severity : np.array(df[df['Severity'] == severity]['Distance(mi)']) for severity in set(df['Severity'])}

part_1_severity = html.Div(
    [
        html.Hr(),
        html.Div(
            [
                html.H2("Part 1: Explore how traffic controls mitigate the impact of a road accident."),
                html.P('''
Some accidents are minor, such as a fender bender in a parking lot.
Other accidents are more severe, and can cause stretches of road to be left unsuitable for driving.

This dataset provides insight into thedistance of impacted road, in miles, of different accidents.
Using this data, we can get insight into how the presence of road controls (such as stop signs)
impact the severity of accidents.

Explore the relation between the distance of road impacted by an accident and the presence of traffic controls.

Satisfies: Box plot, violin plots, statistics.
'''),
                dcc.Checklist(
                    id = "severity-checklist",
                    options = [{'value' : str(x), 'label' : f'{str(x)}'}
                               for x in dist_checklist_keys],
                               #for x in [1, 2, 3, 4]],
                    value   = ['Stop'],
                    labelStyle = {'display' : 'inline-block'}
                ),
                dcc.Graph(id = "severity-distance-boxplot")
            ]
        ),
        html.Hr(),

    ]
)

print("... part 1 callback...")
@app.callback(
    Output("severity-distance-boxplot", "figure"), 
    [Input("severity-checklist", "value")]
)
def part_1_figure(x):
    fig = px.violin(
        df,
        x = x,
        y = 'Distance(mi)',
        box = True,
        points='all'
    )
    return fig

Building part 1...
... part 1 callback...


In [3]:
print("Building part 2...")

weather_keys = ['Temperature(F)', 'Humidity(%)', 'Pressure(in)', 'Visibility(mi)', 'Wind_Speed(mph)', 'Precipitation(in)']

part_2_histogram = html.Div(
    [
        html.Hr(),
        html.Div(
            [
                html.H2("Part 2: Explore weather conditions and traffic accidents."),
                html.P('''
A foggy day, a rainy night, and frozen roads. Inclement weather impacts driving conditions.

Explore the distribution of different weather conditions during traffic accidents.
Does anything seem surprising when you choose different labels and bin sizes?

Satisfies: Kernel Density Estimation, Histogram, Statistics, Correlation Analysis.
                '''),
            ]
        ),
        html.Div(
            [
                dcc.Graph(id = "weather-kde"),
                dcc.Checklist(
                    id = "weather-checklist",
                    options = [
                        {'value' : str(x), 'label' : f'{str(x)}'}
                        for x in weather_keys
                    ],
                    value   = ["Temperature(F)", "Humidity(%)"],
                    labelStyle = {'display' : 'inline-block'}
                ),
                dcc.Slider(
                    id="weather-binsize",
                    min=0.1,
                    max=10.1,
                    value=1.0, 
                    step = 0.1,
                    marks = {
                        x : str(x) 
                        for x in [0.1, 0.2, 0.5, 1.0, 2.0, 5.0, 10.0]
                    }
                )
            ]
        ),
        html.Hr(),    
    ]
)

print("... Part 2 callback...")
@app.callback(
    Output("weather-kde", "figure"), 
    [Input("weather-checklist", "value"),
     Input("weather-binsize", "value")
    ]
)
def part_2_figure(labels, binsize):
    '''
    fig = px.histogram(
        df,
        x = x,
        #marginal = 'violin',
        histnorm = 'probability density',
        nbins = bins
    )
    return fig
    '''
    data = np.nan_to_num(np.array(df[labels]), copy=True, nan=0.0, posinf=0.0, neginf=0.0).T
    fig = ff.create_distplot(data, labels, bin_size=binsize)
    return fig


Building part 2...
... Part 2 callback...


In [4]:
print("Building part 3...")

test_keys = ['Stop', 'Crossing', 'Traffic_Calming', 'Traffic_Signal']
test_metrics = ['Distance(mi)', 'Severity']

part_3_hypothesis = html.Div(
    [
        html.Hr(),
        html.Div(
            [
                html.H2("Part 3: Hypothesis testing"),
                html.P('''
Think back to Part 1, and consider this null hypothesis:
"There is no difference in the distance (or severity) of traffic accidents for those
which occur at stop signs and those which do not."

Intuitively, you might reject this null hypothesis, but with this modal, you can check and be sure.

Assuming alpha = 0.05, explore eight total combinations of hypotheses.
The respective T-scores and P-values are printed.

Satisfies: t-test, p-value, hypothesis testing, box-plot, histogram, kernel density estimation
                '''),
            ]
        ),
        html.Div(
            [
                dcc.Graph(id = "hypothesis-graph"),
                dcc.RadioItems(
                    id = "hypothesis-keys",
                    options = [
                        {'value' : x, 'label' : f"Test how '{x}' ..."}
                        for x in test_keys
                    ],
                    value = "Stop",
                    labelStyle = {'display' : 'inline-block'}
                ),
                dcc.RadioItems(
                    id = "hypothesis-metric",
                    options = [
                        {'value' : x, 'label' : f"... impacts metric '{x}'"}
                        for x in test_metrics
                    ],
                    value = "Distance(mi)",
                    labelStyle = {'display' : 'inline-block'}
                )
            ]
        ),
        html.Hr(),
    ]
)

print("... Part 3 callback...")
@app.callback(
    Output("hypothesis-graph", "figure"), 
    [Input("hypothesis-keys", "value"),
     Input("hypothesis-metric", "value")
    ]
)
def part_3_figure(key = 'Stop', metric = 'Distance(mi)', show_boxes=True):
    # From our dataframe, get items with and without boolean key
    key_with = df[df[key] == True][metric]
    key_without = df[df[key] == False][metric]

    # get degrees of freedom
    v_obs = degreesOfFreedom(key_with, key_without)
    # Get the t-pdf of our two values
    pdf_with = lambda x: stats.t.pdf(
        x, df = v_obs, loc = key_with.mean(), scale = key_with.std()
    )
    pdf_without = lambda x: stats.t.pdf(
        x, df = v_obs, loc = key_without.mean(), scale = key_without.std()
    )
    # get statistics
    t_obs, p_obs = scipy.stats.ttest_ind(key_with, key_without)

    # Lines for graphing
    Xs = np.linspace(-8, 8, 1000)
    Ys_with = [pdf_with(x) for x in Xs]
    Ys_without = [pdf_without(x) for x in Xs]

    # Create figures
    fig = go.Figure()
    fig.add_trace(
        go.Scatter(x=Xs, y=Ys_with, mode='lines', name=f"With {key}")
    )
    fig.add_trace(
        go.Scatter(x=Xs, y=Ys_without, mode='lines', name=f"Without {key}")
    )
    # Add text
    fig.add_trace(
        go.Scatter(
            x = [0],
            y = [0.50],
            text = [f"T-stat: {t_obs:.3f}    P-score: {p_obs:.3f}"],
            mode = "text",
            name = "Stats"
        )
    )
    
    if show_boxes:
        fig.add_trace(
            go.Box(x=key_with, name=f"Boxplot accidents with {key}")
        )
        fig.add_trace(
            go.Box(x=key_without, name=f"Boxplot  accidents without {key}")
        )
    
    return fig

Building part 3...
... Part 3 callback...


In [5]:
print("Building part 4...")

part_4_map = html.Div(
    [
        html.Div(
            [
                html.H2("Part 4: Map of accidents by severity"),
                html.P('''
The dataset labels accidents by the severity of the imapct it has on traffic,
on a scale of 1 (not severe) to 4 (most severe). Explore the distribution of
severity of these accidents here, and how they relate with different traffic measures.

Satisfies: Cartogram map, geospatial analysis
                '''),
            ]
        ),
        html.Div(
            [
                dcc.Graph(id = "accidents-map"),
                dcc.RadioItems(
                    id = "map-severity",
                    options = [{'value' : x, 'label' : f"Severity {x}"}
                               for x in [1,2,3,4]],
                    value = 1,
                    labelStyle = {'display' : 'inline-block'}
                ),
                dcc.RadioItems(
                    id = "severity-checklist-map",
                    options = [{'value' : str(x), 'label' : f'{str(x)}'}
                               for x in ['Any'] + dist_checklist_keys],
                    value   = 'Any',
                    labelStyle = {'display' : 'inline-block'}
                ),
            ]
        )
                
    ]
)

print("... Part 4 callback...")
@app.callback(
    Output("accidents-map", "figure"), 
    [Input("map-severity", "value"),
     Input("severity-checklist-map", "value"),
    ]
)
def part_4_figure(severity = [1], traffic_measure = 'Any'):
    colors = [
        'rgba(0,0,0,0)',
        'rgba(0,128,192,192)',
        'rgba(128,0,192,192)',
        'rgba(256,0,128,192)', 
        'rgba(256,128,0,192)'
    ]
    
    if traffic_measure == 'Any':
        df_subset = df[df['Severity'] == severity]
    else:
        df_subset = df[df['Severity'] == severity][df[traffic_measure] == True]
    
    color_str = colors[severity]
    color = [color_str for _ in range(len(df_subset))]
    
    fig = go.Figure(
        data = go.Scattergeo(
            lat=df_subset['Start_Lat'],
            lon=df_subset['Start_Lng'],
            marker_color = color,
            marker_size = 4,
        )
    )
    
    fig.update_layout(geo_scope='usa')
    
    return fig

Building part 4...
... Part 4 callback...


In [6]:
print("Preparing part 5...")

K_CLUSTERS = 12
hues = [get_hues(ii) for ii in range(300)]
all_geo_data = np.array([df_full["Start_Lng"], df_full["Start_Lat"]]).T
some_geo_data = np.array([df["Start_Lng"], df["Start_Lat"]]).T

print("... fitting for part 5...")
gmm = GaussianMixture(n_components = K_CLUSTERS)
gmm.fit(get_sample(all_geo_data, 200_000))
print("... Fitting done!")

Preparing part 5...
... fitting for part 5...
... Fitting done!


In [7]:
print("... generating part 5 figure...")

part_5_labels = gmm.predict(some_geo_data)
part_5_colors = [hues[index] for index in part_5_labels]

def part_5_figure():
    fig = go.Figure(
            data = go.Scattergeo(
                lat=df['Start_Lat'],
                lon=df['Start_Lng'],
                marker_color = part_5_colors,
                marker_size = 2,
            )
        )
    fig.update_layout(geo_scope='usa')
    return fig

part_5_figure_instance = part_5_figure()
print("... Figure generated.")

... generating part 5 figure...
... Figure generated.


In [8]:
print("... building part 5...")

part_5_map = html.Div(
    [
        html.Hr(),
        html.Div(
            [
                html.H2("Part 5: Accident clustering"),
                html.P('''
This modal plots the same map as above, but now with all points clustered according to location.

This uses a 12-class Gaussian Mixture Model.

Satisfies: Gaussian mixture model, classification, clustering.

Note that we skip K-Means and hierarchical clustering,
since it is too intensive to display, even on this subset of 10000 points.
                '''),
                dcc.Graph(id = "clustered-map", figure = part_5_figure_instance)
            ]
        ),
        html.Hr()
    ]
)

... building part 5...


In [9]:
print("Preparing part 6 data...")

df_real = df.copy().replace(
    [False, True, -np.inf, np.inf, np.nan],
    [0.00,  1.00,  0.0,  0.0,  0.0]
)

axis_keys = [
    'Distance(mi)', 'Severity',
    'Stop', 'Crossing', 'Traffic_Calming', 'Traffic_Signal',
    'Temperature(F)', 'Humidity(%)', 'Pressure(in)', 'Visibility(mi)','Wind_Speed(mph)', 'Precipitation(in)',
]

print("... Preparing part 6 tools...")
def eval_polyfit(xx, coefs):
    '''
    For given x, and coefficients from np.polynomial.polynomial.polyfit(X,Y)
    '''
    s = 0
    for ii, coef in enumerate(coefs):
        s += xx**ii * coef
    
    return s

def get_polyfit_points(
    dataframe = df_real, xkey = 'Stop', ykey = 'Distance(mi)', deg = 3
):
    # 1. Get coefficients 
    polyfit = np.polynomial.polynomial.polyfit(
        dataframe[xkey],
        dataframe[ykey],
        deg = deg
    )
    
    # 2. Get Xrange
    Xs = np.linspace(dataframe[xkey].min(), dataframe[xkey].max(), 1000)
    
    # 3. Get Ys
    Ys = [eval_polyfit(xx, polyfit) for xx in Xs]
    
    return Xs, Ys, polyfit

Preparing part 6 data...
... Preparing part 6 tools...


In [10]:
print("... Building part 6.")

part_6_fitting = html.Div(
    [
        html.Hr(),
        html.Div(
            [
                html.H2("Final part: Arbitrary regressions"),
                html.P('''
In this final modal, please explore combinations of all the data items.
This modal will allow you to scatter (along X and Y axes) any two items from the dataset.

This modal also includes regression fitting, with user-set exponent (linear, quadratic, etc.)

Satisfies: Line graph, basics, linear regression, pearson correlation, correlation analysis.
                '''),
                html.Div(
                    [
                        dcc.Graph(id="fitting-graph"),
                    ],
                    style={'display': 'inline-block', 'vertical-align': 'left'}
                ),
                html.Div(
                    [
                        dcc.Dropdown(
                            id = 'fit-xaxis',
                            options = [
                                {'value' : x, 'label' : x}
                                for x in axis_keys
                            ],
                            value = 'Precipitation(in)',
                        ),
                        dcc.Dropdown(
                            id = 'fit-yaxis',
                            options = [
                                {'value' : x, 'label' : x}
                                for x in axis_keys
                            ],
                            value = 'Severity',
                        ),
                        dcc.Dropdown(
                            id = 'fit-exponent',
                            options = [
                                {'value' : 1, 'label' : "Linear (x¹)"},
                                {'value' : 2, 'label' : "Quadratic (x²)"},
                                {'value' : 3, 'label' : "Cubic (x³)"},
                                {'value' : 4, 'label' : "... (x⁴)"},
                                {'value' : 5, 'label' : "... (x⁵)"}
                            ],
                            value = 3,
                        ),
                    ],
                    style={'width': '15%', 'display': 'inline-block', 'vertical-align': 'top'}
                )
            ]
        ),
        html.Hr(),
    ]
)

print("Part 6 callback...")
@app.callback(
    Output("fitting-graph", "figure"),
    [
        Input("fit-xaxis", "value"),
        Input("fit-yaxis", "value"),
        Input("fit-exponent", "value"),
    ]
)
def get_figure_part6(xkey = 'Stop', ykey = 'Distance(mi)', deg = 1):
    dataframe = df_real
    print(xkey, ykey, deg)
    fig = px.scatter(dataframe, xkey, ykey) #go.Figure()

    # 2. Plot fit
    Xs, Ys, coefs = get_polyfit_points(dataframe, xkey, ykey, deg = deg)
    fig.add_trace(
        go.Scatter(x = Xs, y = Ys, mode='lines', name = "Regression")
    )

    # 3. Get Pearson correlation
    pcc = np.corrcoef(dataframe[xkey], dataframe[ykey])[0,1]

    # 4. Plot coefficients and correlation
    midpoint_idx = len(Xs)//2

    funcstr = "y = "
    power   = len(coefs) - 1
    for coef in coefs[::-1]:
        funcstr = funcstr + str(np.round(coef, 2)) + "x^" + str(power) + " + "
        power = power - 1

    funcstr = funcstr[:-5] # remove '^0 + '

    fig.add_trace(
        go.Scatter(
            x = [Xs[midpoint_idx],],
            y = [dataframe[ykey].max()] ,
            text = [funcstr + f"; with Pearson ρ: {pcc:.3f}",],
            mode = "text",
            name = "Fitting parameters"
        )
    )

    return fig

... Building part 6.
Part 6 callback...


In [ ]:
print("Hooray! Finished building all the individual parts.")

print("Building video...")
part_video = 
html.Div(
    [
        html.Video(
            controls = True,
            id = 'movie_player',
            src = "./static/demo_video.mp4",
            autoPlay=False,
            width = 1024
        ),
        html.H2("US Accidents Analysis dashboard: Feb 2016 to December 2020")
        html.P('''
This dashboard uses [Sobhan Moosavi's](https://www.kaggle.com/sobhanmoosavi) Kaggle dataset,
["US Accidents"](https://www.kaggle.com/sobhanmoosavi/us-accidents).

This dataset compiles US traffic accidents from 49 US states. It includes weather information and traffic conditions from the time and place each accident takes place.

This dataset also records the **distance** of road impacted by the accident (in miles), and the **severity** of the impact on traffic flor (from 1 to 4.)
'''
              )
    ]
)

print("Building final layout...")
app.layout = html.Div(
    [
        dcc.Tabs(
            id = "studio-tabs",
            value = "tab-video",
            children = [
                dcc.Tab(
                    label = "Demo video",
                    value = "tab-video"
                ),
                dcc.Tab(
                    label = "Controls vs Severity",
                    value = "tab-1"
                ),
                dcc.Tab(
                    label = "Road conditions",
                    value = "tab-2"
                ),
                dcc.Tab(
                    label = "Testing Hypotheses",
                    value = "tab-3"
                ),
                dcc.Tab(
                    label = "Accidents map",
                    value = "tab-4"
                ),
                dcc.Tab(
                    label = "Clustering accidents",
                    value = "tab-5"
                ),
                dcc.Tab(
                    label = "Regression studio",
                    value = "tab-6"
                ),
            ]
        ),
        html.Div(id='current-content')
    ],
    style={
        'width': '1024px',
        'margin' : 'auto',
        
    }
)

print("Building pages dict...")
pages = {
    'tab-video' : part_video,
    'tab-1'     : part_1_severity,
    'tab-2'     : part_2_histogram,
    'tab-3'     : part_3_hypothesis,
    'tab-4'     : part_4_map,
    'tab-5'     : part_5_map,
    'tab-6'     : part_6_fitting
}

print("Building final callback...")
@app.callback(Output('current-content', 'children'),
              Input('studio-tabs', 'value'))
def get_tab_content(tab_string):
    print(tab_string)
    return pages[tab_string]

print("Bada bing bada boom! Now all that is left is to run the server...")
app.run_server(port = 8087, debug=False)
print("... The end! Bye bye.")

Hooray! Finished building all the individual parts.
Building video...
Building final layout...
Building pages dict...
Building final callback...
Bada bing bada boom! Now all that is left is to run the server...
Dash is running on http://127.0.0.1:8087/

 * Serving Flask app 'Traffic accident data studio' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8087/ (Press CTRL+C to quit)
127.0.0.1 - - [12/Dec/2021 20:58:13] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [12/Dec/2021 20:58:13] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [12/Dec/2021 20:58:13] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [12/Dec/2021 20:58:13] "GET /_favicon.ico?v=2.0.0 HTTP/1.1" 200 -
127.0.0.1 - - [12/Dec/2021 20:58:13] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [12/Dec/2021 20:58:13] "GET /static/demo_video.mp4 HTTP/1.1" 206 -
127.0.0.1 - - [12/Dec/2021 20:58:13] "GET /static/demo_video.mp4 HTTP/1.1" 206 -
127.0.0.1 - - [12/Dec/2021 20:58:13] "GET /static/demo_video.mp4 HTTP/1.1" 206 -
127.0.0.1 - - [12/Dec/2021 20:58:13] "GET /static/demo_video.mp4 HTTP/1.1" 206 -


tab-video


127.0.0.1 - - [12/Dec/2021 20:58:14] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [12/Dec/2021 20:58:14] "GET /_dash-component-suites/dash/dcc/async-graph.js HTTP/1.1" 200 -
127.0.0.1 - - [12/Dec/2021 20:58:14] "GET /_dash-component-suites/dash/dcc/async-dropdown.js HTTP/1.1" 200 -
127.0.0.1 - - [12/Dec/2021 20:58:14] "GET /_dash-component-suites/dash/dcc/async-plotlyjs.js HTTP/1.1" 200 -


tab-6
Precipitation(in) Severity 3


...

Stop Stop 3


127.0.0.1 - - [12/Dec/2021 21:51:33] "POST /_dash-update-component HTTP/1.1" 200 -


Stop Distance(mi) 3


127.0.0.1 - - [12/Dec/2021 21:51:41] "POST /_dash-update-component HTTP/1.1" 200 -


Stop Distance(mi) 5
